In [2]:
import numpy
import pandas as pd
import nltk
from ast import literal_eval
import treetaggerwrapper as ttw
from nltk.stem.porter import *

In [3]:
recipes = pd.read_csv("data/RAW_recipes.csv")
recipes["steps"] = recipes["steps"].apply(literal_eval)
recipes["ingredients"] = recipes["ingredients"].apply(literal_eval)
recipes["tags"] = recipes["tags"].apply(literal_eval)

In [78]:
recipes["tags"][1]

['30-minutes-or-less',
 'time-to-make',
 'course',
 'main-ingredient',
 'cuisine',
 'preparation',
 'occasion',
 'north-american',
 'breakfast',
 'main-dish',
 'pork',
 'american',
 'oven',
 'easy',
 'kid-friendly',
 'pizza',
 'dietary',
 'northeastern-united-states',
 'meat',
 'equipment']

In [4]:
recipes["steps"][0]

['make a choice and proceed with recipe',
 'depending on size of squash , cut into half or fourths',
 'remove seeds',
 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece',
 'season with mexican seasoning mix ii',
 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece',
 'season with sweet mexican spice mix',
 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin',
 'be careful not to burn the squash especially if you opt to use sugar or butter',
 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking',
 'if desired , season with salt']

In [5]:
recipes["ingredients"][0]

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt']

In [7]:
preprocessed_sentence = ''
lemma_sentence = ''
def tree_tag(text, tagger):
    tagged_text = tagger.tag_text(text)
    return ttw.make_tags(tagged_text)

In [22]:
def tag_ingredients(ingredients):
    
    tagged_ingredients = []
    for ingredient in ingredients:
    
        sentence = tree_tag(ingredient, tagger)

        for tag in sentence:
        # remove special characters
            if isinstance(tag, ttw.Tag):

                # stem
                stem = stemmer.stem(tag.lemma)
                
                if tag.pos in ['NN0', 'NN1', 'NN2']:
                    tagged_ingredients.append(stem.split('|')[0])
    return tagged_ingredients


In [63]:
def tag_sentence(original_sentence, ingredients):
    
    sentence = tree_tag(original_sentence, tagger)

    step_ingredients = []
    step_activities = []
    
    for tag in sentence:
    # remove special characters
        if isinstance(tag, ttw.Tag):
            
            # stem
            stem = stemmer.stem(tag.lemma)
            if stem in ingredients:
                step_ingredients.append(tag.lemma.split('|')[0])
            if tag.pos in ['VVB','VVZ','VVI',
                           'VVD','VVN', 'VVP','VVG']:#VVG not sure
                step_activities.append(tag.lemma.split('|')[0])
                
    if len(step_activities) == 0:
        # default activity if no verb is present
        step_activities.append('use')

    return step_ingredients, step_activities

            

In [91]:
tagger = ttw.TreeTagger(TAGLANG='en')
stemmer = PorterStemmer()

case_id_col = []
event_id_col = []
order_col = []
activity_col = []
ingredient_col = []

event_cnt = 0 

for i in range(0,recipes.shape[0]):

    if 'cocktails' in recipes["tags"][i]:
        ingredients = tag_ingredients(recipes["ingredients"][i])

        order_cnt = 0

        for original_sentence in recipes["steps"][i]:
            step_ingredients, step_activities = tag_sentence(original_sentence,ingredients)

            # synchronous events
            for ingredient in step_ingredients:
                for activity in step_activities:
                    case_id_col.append(i)
                    event_id_col.append(event_cnt)
                    event_cnt += 1
                    order_col.append(order_cnt)

                    activity_col.append(activity)
                    ingredient_col.append(ingredient)

            order_cnt += 1

In [92]:
recipe_log = pd.DataFrame({'case_id': case_id_col,
                           'event_id': event_id_col,
                           'order': order_col,
                           'activity': activity_col,
                           'ingredient': ingredient_col})
recipe_log

,case_id,event_id,order,activity,ingredient
0,102,0,0,use,limeade
1,102,1,0,use,lime
2,102,2,0,use,juice
3,102,3,0,use,tequila
4,102,4,2,garnish,lime
...,...,...,...,...,...
25252,230329,25252,1,add,juice
25253,230329,25253,2,fill,soda
25254,230382,25254,0,mix,ice
25255,230382,25255,1,garnish,pickle


In [93]:
recipe_log.to_csv('log_cocktails.csv',index = False)